<h4><i><font color='red'>The project has many components, and we encourage you to complete as many as you can! That said, we highly encourage you to submit your work even if your notebook is only partially completed - the TA can help review your work and provide tips on any places that you got stuck or have further questions!</font></i></h4>

<font size="6">**Table of Contents:**</font>  
<br>
<font size="5">
<u>Behavioral Metrics:</u>  
1\. [Introduction](#scrollTo=mKT-yDAzlP9Y&uniqifier=1)   
2\. [Setup](#scrollTo=SuQ7Xnp1lolE&uniqifier=1)   
3\. [Traditional metrics: NDCG@K](#scrollTo=H1_DJMNDoW-5&uniqifier=1)   
4\. [TODO 4.1: Implement 3 Behavioral Metrics](#scrollTo=rkHkC-SnsQRI&uniqifier=1)   
5\. [TODO 4.2: Implement a New, Non-Learnt Ranker](#scrollTo=a4ZcoFf8uL2v&uniqifier=1)   
<br>
<u>Off-Policy Evaluation:</u>  
1\. [Introduction](#scrollTo=LKg4HQSbxwCY&uniqifier=1)   
2\. [Setup](#scrollTo=Qp_3_z7myCKU&uniqifier=1)   
3\. [Rewards and IPS](#scrollTo=9RboprQXyuCB&uniqifier=1)   
4\. [TODO 4.3: Capped IPS and NCIS](#scrollTo=zQuEVahrzNV9&uniqifier=1)   
</font>



# <u>Behavioral Metrics:</u>

## **1. Introduction**
[back to top](#scrollTo=bvONhqQblFfl&uniqifier=1)

Welcome to the last week of the course -- so excited to see that you've made it to the end! 👏 

We've already discussed the importance of measuring model performance. As Lord Kelvin said, "To measure is to know – If you cannot measure it, you cannot improve it." And he was right – metrics are the only way we can actually evaluate our model’s performance!

In this week's project, we will touch upon two key aspects related to evaluation:
1. Behavioral metrics: We will work with a Spotify music sessions dataset and implement a few behavioral metrics to understand their relationships with traditional metrics.
2. Off-policy evaluation: We will simulate a dataset where we have logged action policies, and see how IPS is implemented.
  
<br>

### About the Data: Spotify Music Sessions
Behavioral metrics include factors like what items a user interacts with and how, the amount of time they spend on the platform, and how they spend that time. To define and implement a few behavioral metrics, we will work with the Spotify music sessions dataset.

The public part of the dataset consists of roughly 130 million listening sessions with associated user interactions on the Spotify service. In total, users interacted with almost 4 million tracks during these sessions, and the dataset includes acoustic features and metadata for all of these tracks. We'll be working with a sample of the full dataset.
  
<br>A detailed description of the dataset can be found [here](https://drive.google.com/file/d/1BELTuH4nBeyHna5EAGzJv-HWHKrbxPsf/view?usp=sharing).


## **2. Setup**
[back to top](#scrollTo=bvONhqQblFfl&uniqifier=1)

In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.metrics import ndcg_score
from collections import OrderedDict
from IPython.display import clear_output
from functools import partial
from tqdm import tqdm
tqdm.pandas()


# Download the data
!gdown --folder https://drive.google.com/drive/folders/10LGZMgXRuz2qPr_QDbYdVVlKEcnQ25YL?usp=sharing -O ./
clear_output()

In [ ]:
# Read in data
log = pd.read_csv("data/log_mini.csv")
tracks = pd.read_csv("data/tf_mini.csv")

log.shape, tracks.shape

((167880, 21), (50704, 30))

## **3. Traditional metrics: NDCG@K**
[back to top](#scrollTo=bvONhqQblFfl&uniqifier=1)

Normalized Discounted Cumulative Gain (NDCG) is a metric used to evaluate a ranked list of candidates, which values both order and relevancy of the items. In other words, it is able to use the fact that some documents are "more" relevant than others. Highly relevant items should come before medium relevant items, which should come before non-relevant items.
  
<br>We will use the log dataframe as the main dataframe for evaluation of metrics. The `skip_1` flag can be used as a relevance signal -- if the user found the recommendation relevant, `skip_1 = False`. With this relevance signal, we can compute simple NDCG metrics -- one for each session and then averaged across all sessions. This will serve as a base metric for comparison.

Note: the ranking logic here is assumed to be the production ranker, i.e. sorting by `session_position` gives the exact order of tracks the Spotify ranker presented to the user.
  
<br>Let's compute a simple skip rate and NDCG metric for the production ranker:

In [ ]:
# Compute average skip rate @ 10

topk = 10
log.sort_values(by=['session_position']).groupby("session_id").apply(lambda x: np.mean(x['skip_1'].values[:topk])).mean()

0.40894

In [ ]:
# Compute NDCG @ 10

def compute_ndcg(df, topk):
    df = df.iloc[:topk]
    true_relevance = np.asarray(1-np.asarray(df['skip_1']*1.0))
    ranker_scores = np.asarray(1/(np.asarray(range(len(df)))+1)) # Approximate the ranker scores using the session position
    return ndcg_score([true_relevance], [ranker_scores])

topk = 10
log.sort_values(by=['session_position']).groupby("session_id").progress_apply(partial(compute_ndcg, topk=topk)).mean()

100%|██████████| 10000/10000 [00:08<00:00, 1163.95it/s]


0.8330266041453142

## **4. TODO 4.1: Implement 3 Behavioral Metrics**
[back to top](#scrollTo=bvONhqQblFfl&uniqifier=1)

Your first goal is to implement three behavioral metrics and compare their correlations:
1. *Time to first skip:* How long did it take for the user to get the first bad recommendation, i.e. a recommendation they skipped. Since we can't easily calculate time, we can use number of songs as a proxy and compute the metric as number of songs it took for the first skip.

2. *Sustained dissatisfaction:* We assume that the user is dissatisfied in a sustained manner if they skip 3 songs consecutively.

3. *Session coherence:* We define coherence as how similar the recommended musical tracks are. We can use the acoustic_vector of the music tracks to calculate the similarity.

Once these metrics are implemented, compute them for `topk=5` and `topk=10`. Compare their estimates for the production ranker as a correlation plot. Please note which metrics are correlated with NDCG.

In [ ]:
# Implement session metric 1:
    # Time to first skip (number of songs to first skip)
    # Our implementation found the answer to be 4.6380
        # We assumed no skips yields the number of songs listened to as opposed to a null

In [ ]:
# Implement session metric 2:
    # Sustained dissatisfaction: Proportions of sessions with 3 consecutive skips
    # Our implementation found the answer to be 0.6543

In [ ]:
# Implement session metric 3: 
    # Session coherence: Average similarity between the top recommended tracks
    # Our implementation found the answer to be ~0.8097
        # We calculated the Cosine Similarity between tracks using the 8 element acoustic vector per track
        # Cosine Similarity is the dot product of unit-length vectors
        # Feel free to use a different similarity metric, such as Euclidean Distance from last week

In [ ]:
# Correlation plot amongst metrics and NDCG


## **5. TODO 4.2: Implement a New, Non-Learnt Ranker**
[back to top](#scrollTo=bvONhqQblFfl&uniqifier=1)

Your next goal is to implement another simple, non-learnt ranking logic, and compare the performance of both the production ranker and new ranking policy on the NDCG and three behavioral metrics you created above. A simple ranking policy could include sort by track popularity, or sort by danceability score for the track.
  
<br>Implement your ranker and then comment on its performance on our behavioral metrics compared to Spotify's production ranker:

# <u>Off-Policy Evaluation:</u>

## **1. Introduction**
[back to top](#scrollTo=bvONhqQblFfl&uniqifier=1)

We log listener behavior based on the recommendations that the production recommender serves to the listener. Using this data to assess any new recommender system, however, can present challenges – the production recommender and the new recommender can drastically differ in the results that they display to the user. For example, maybe the new recommender presents a lot of niche content, while the production recommender presents a lot of popular options. This can be an issue when evaluating a new recommender – If you don’t have any feedback on a recommendation because you never presented it to a user, how can you evaluate whether it’s a good recommendation?
If you have a new policy to test that’s very similar to your old approach, then this won’t be an issue, and it’ll be easy to test! However, if the policy is very different, then you’ll need to collect special logged data.

In this part of the project, we will simulate a recommendation policy and leverage counterfactual estimators as metrics to compare performance.

## **2. Setup**
[back to top](#scrollTo=bvONhqQblFfl&uniqifier=1)

Let's first begin by generating a few users and products. For ease of simulation, we assume users derive equal satisfaction from each item:

In [ ]:
users = np.array(["user1", "user2", "user3"])

products = np.array(
    [
        "product_a",
        "product_b",
        "product_c",
        "product_d",
        "product_e",
        "product_f",
        "product_g",
    ]
)

satisfaction = {
    "product_a": 100,
    "product_b": 150,
    "product_c": 100,
    "product_d": 200,
    "product_e": 500,
    "product_f": 120,
    "product_g": 160,
}

Let's also implement whether a given user will accept a given recommendation or not. Once done, we can implement a target policy that makes recommendations.

In [ ]:
def will_purchase(user, product):
    if user == "user1" and (
        product == "product_a" or product == "product_b" or product == "product_c"
    ):
        return True
    elif user == "user2" and (product == "product_d" or product == "product_e"):
        return True
    elif user == "user3" and (product == "product_f" or product == "product_g"):
        return True
    else:
        return False


def choose_user():
    return np.random.choice(users, size=1)


def logging_policy():
    return np.random.choice(products, size=1), 1 / len(products)


class TargetPolicy:
    def __init__(self):
        self.user_probs = {
            "user1": np.array([0.1, 0.1, 0.2, 0.1, 0.15, 0.15, 0.20]),
            "user2": np.array([0.1, 0.10, 0.05, 0.25, 0.3, 0.1, 0.1]),
            "user3": np.array([0.06, 0.06, 0.3, 0.06, 0.06, 0.4, 0.06]),
        }

        for user, probs in self.user_probs.items():
            assert probs.sum() == 1
            assert len(probs) == len(products)

    def recommend(self, user):
        user_prob = self.user_probs[user]
        product = np.random.choice(products, size=1, p=user_prob)
        product_idx = np.where(products == product)
        prob = user_prob[product_idx]

        return product, prob

    def get_prob(self, user, product):
        user_prob = self.user_probs[user]
        product_idx = np.where(products == product)
        product_prob = user_prob[product_idx]

        return product_prob

Having defined all key components of the dataset generation, let's create logged data that we can finally use for evaluation purposes:

In [ ]:
def compute_satisfaction(user, product):
    if will_purchase(user, product):
        return satisfaction[product.item()]
    else:
        return 0


def create_logs(n=1000):
    logs = []
    target_policy = TargetPolicy()

    for _ in tqdm(range(n)):
        user = choose_user()

        logging_product, logging_prob = logging_policy()
        model_prob = target_policy.get_prob(user.item(), logging_product)

        target_product, _ = target_policy.recommend(user.item())

        logging_satisfaction = compute_satisfaction(user, logging_product)
        target_satisfaction = compute_satisfaction(user, target_product)

        log = OrderedDict(
            {
                "user_features": user.item(),
                "item_placed": logging_product.item(),
                "item_prob": logging_prob,
                "item_satisfaction": logging_satisfaction,
                "model_prob": model_prob.item(),
                "ab_test_satisfaction": target_satisfaction,
            }
        )

        logs.append(log)

    return pd.DataFrame(logs)

Here is what our logged data now looks like:

In [ ]:
logs = create_logs(n=1000)
logs.head(5)

100%|██████████| 1000/1000 [00:00<00:00, 7306.50it/s]


,user_features,item_placed,item_prob,item_satisfaction,model_prob,ab_test_satisfaction
0,user3,product_b,0.142857,0,0.06,0
1,user3,product_c,0.142857,0,0.30,0
2,user3,product_b,0.142857,0,0.06,120
3,user2,product_b,0.142857,0,0.10,0
4,user1,product_c,0.142857,100,0.20,0


## **3. Rewards and IPS**
[back to top](#scrollTo=bvONhqQblFfl&uniqifier=1)

With the dataset ready, let's compute the mean rewards (satisfaction) for the logging/production policy and the target policy. Note that due to randomness in the log generation process your result will likely differ.


In [ ]:
sim = create_logs(n=100_000)
logging_policy_reward = sim["item_satisfaction"].mean()
target_policy_reward = sim["ab_test_satisfaction"].mean()

print(f"Expected reward from logging policy: {logging_policy_reward:.2f}")
print(f"Expected reward from target policy: {target_policy_reward:.2f}")

100%|██████████| 100000/100000 [00:13<00:00, 7446.50it/s]


Expected reward from logging policy:  62.48
Expected reward from target policy:  100.71


Now let's implement the IPS estimator:

In [ ]:
def compute_ips(df):
    assert {"model_prob", "item_prob", "item_satisfaction"}.issubset(df.columns)
    return (df["model_prob"] / df["item_prob"] * df["item_satisfaction"]).mean()

compute_ips(logs)

101.51120000000002

Computing the IPS estimator on our 1,000 entry log gives an average revenue much closer to our expected reward from the target policy compared to that of the logging policy. Therefore, we should be confident to deploy our target policy to production and do an A/B test comparing it with the logging policy as a final validation.

## **4. TODO 4.3: Capped IPS and NCIS**
[back to top](#scrollTo=bvONhqQblFfl&uniqifier=1)

Your goal is to implement two additional off-policy estimators:
1. Capped IPS
2. Normalized Capped Importance Sampling (NCIS)

Feel free to try different capping thresholds, and compare the reward and standard deviations of these estimators with the IPS estimator and mean reward.

In [ ]:
def compute_capped_ips(logs, cap=1000):
    """
    Computes the Capped IPS.

    Args:
        logs (DataFrame): Generated logs data
        cap (int): Capping threshold

    Returns:
        ips (float): Resulting IPS
    """
    ips = 0

    # Your code goes here


    return ips

In [ ]:
def compute_ncis(logs, cap=1000):
    """
    Computes the NCIS.

    Args:
        logs (DataFrame): Generated logs data
        cap (int): Capping threshold

    Returns:
        ncis (float): Resulting NCIS
    """
    ncis = 0

    # Your code goes here


    return ncis